# Hand localization model training with YOLOv5
In this notebook we will generate a model for the hand recognition and localization task using YOLOv5 and train it.
The procedure is divided into steps:
1) Manage dependencies and main YOLOv5 files
2) Download and extract dataset
3) Check on from some random samples that conversion is good
4) Last steps
5) Train the model

### 1) Download YOLO and import necessary dependencies

In [1]:
import os, shutil
import random
if not os.path.isdir('yolov5'):
    os.system('git clone https://github.com/ultralytics/yolov5')

# install and import yolov5 dependencies
!pip install -qr yolov5/requirements.txt
import torch, random
import numpy as np
from IPython.display import Image, clear_output

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# import dependencies to extract dataset
import requests, tarfile, glob

# import & install dependencies needed to convert dataset label format to yolov5
!apt-get install libmagic-dev
!pip install python-magic
import magic, re, scipy.io

# import libs to verify conversion in step 4
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

Cloning into 'yolov5'...


Setup complete. Using torch 1.12.0+cu116 (Quadro RTX 5000)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libmagic-dev
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/pyt

### 2) Download and extract dataset
We now check for the presence of the three datasets, if the program doesn't find them then it will procede to download them.
We also clone a support repository created to supply this notebook with all the annotations alredy converted in the yolov5 notation (converting the annotation in the notebook would be an unneccessary extra step),
and the HandsOverFace dataset alredy extracted and cleared from images used in the provided evaluation dataset.
We chose to upload HandsOverFace into a repository not for the fact that is easier, but because it was not possible for us to unzip it in the notebook.

In [2]:
# check if datasets have been downloaded: download it otherwise

# First we import the oxford dataset
oxford_path = "hand_dataset.tar.gz"
if not(os.path.exists(oxford_path)):
    response = requests.get("https://www.robots.ox.ac.uk/~vgg/data/hands/downloads/hand_dataset.tar.gz")
    open(oxford_path, "wb").write(response.content)

# Then the egohands dataset
egohands_path = 'egohands_data.zip'
if not(os.path.exists(egohands_path)):
  response = requests.get('http://vision.soic.indiana.edu/egohands_files/egohands_data.zip')
  open(egohands_path, 'wb').write(response.content)

# And finally we import the hands_over_face dataset, plus all the labels we previously converted
repo_path = 'dataset_CV'
if not(os.path.exists(repo_path)):
    !git clone https://github.com/afParadox/dataset_CV.git

Cloning into 'dataset_CV'...
remote: Enumerating objects: 19645, done.
remote: Counting objects: 100% (3800/3800), done.
remote: Compressing objects: 100% (3732/3732), done.
remote: Total 19645 (delta 4), reused 3794 (delta 2), pack-reused 15845
Receiving objects: 100% (19645/19645), 1.23 GiB | 21.54 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (10105/10105), done.


In the next cell we create the dataset directory, where we'll unzip all the files and reorganize them into training, validation and test.

In [3]:
if not(os.path.exists('dataset/')):
    !mkdir dataset dataset/images/ dataset/labels/ dataset/egohands
    !tar -xvzf hand_dataset.tar.gz -C dataset/
    !unzip egohands_data.zip -d dataset/egohands

Now we organize the three datasets that we downloaded into a single one, removing all non-necessary files, and renaming the files when necessary, as in the case of the egohands dataset.
The photos included in the evaluation dataset provided by the professor get removed from the dataset used in these cells.
In the case of the egohands dataset we also get rid of the whole folders where the photos from the evaluation dataset have been taken. From the total of 48 subfolders in the dataset, only 39 are been used for the training.

We set as final_path the directory dataset/images/ since all images of the datasets will be moved there.

In [4]:
final_path = 'dataset/images/'

Because of the problematic names of the photos of the EgoHands dataset we are forced to rename them all. For every folder we start naming the files from a multiple of 1000.
We are also careful to just move the folders that don't include an image that is used in the evaluation dataset provided by the professor.

In [5]:
# egohands
path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_COURTYARD_B_T/'
counter = 1000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_COURTYARD_H_S/'
counter = 2000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_COURTYARD_S_H/'
counter = 4000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_COURTYARD_T_B/'
counter = 3000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_H_S/'
counter = 5000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_S_H/'
counter = 6000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_LIVINGROOM_T_B/'
counter = 7000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_OFFICE_S_B/'
counter = 8000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CARDS_OFFICE_T_H/'
counter = 9000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_COURTYARD_H_S/'
counter = 10000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_COURTYARD_S_H/'
counter = 11000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_COURTYARD_T_B/'
counter = 12000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_B_S/'
counter = 13000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_H_T/'
counter = 14000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_S_B/'
counter = 15000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_LIVINGROOM_T_H/'
counter = 16000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_OFFICE_H_T/'
counter = 17000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_OFFICE_S_B/'
counter = 18000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/CHESS_OFFICE_T_H/'
counter = 19000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_H_B/'
counter = 20000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_S_T/'
counter = 21000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_COURTYARD_T_S/'
counter = 22000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_LIVINGROOM_B_H/'
counter = 23000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_LIVINGROOM_S_T/'
counter = 24000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_LIVINGROOM_T_S/'
counter = 25000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_OFFICE_B_S/'
counter = 26000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_OFFICE_H_T/'
counter = 27000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_OFFICE_S_B/'
counter = 28000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/JENGA_OFFICE_T_H/'
counter = 29000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_H_T/'
counter = 30000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_S_B/'
counter = 31000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_COURTYARD_T_H/'
counter = 32000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_LIVINGROOM_B_T/'
counter = 33000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_LIVINGROOM_H_S/'
counter = 34000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_LIVINGROOM_S_H/'
counter = 35000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_LIVINGROOM_T_B/'
counter = 36000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_H_B/'
counter = 37000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_S_T/'
counter = 38000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

path = 'dataset/egohands/_LABELLED_SAMPLES/PUZZLE_OFFICE_T_S/'
counter = 39000
for file in sorted(glob.glob(path + '*.jpg')):
  new_name = path + str(counter) + '.jpg'
  print(file + ' -> ' + new_name)
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

!rm -rf dataset/egohands/

In the case of the oxford dataset we don't have to change every image name, but only the ones in the test_data subdirectory, since their names conflict with other files in the dataset.
Also, we have to check each folder for the extra files that we don't need.

In [6]:
# oxford 
path = 'dataset/hand_dataset/training_dataset/training_data/images/'
for file in os.listdir(path):
  if file == '._.DS_Store' or file == '.DS_Store': continue
  os.rename(path + file, final_path + file)

path = 'dataset/hand_dataset/validation_dataset/validation_data/images/'
for file in os.listdir(path):
  if file == '._.DS_Store' or file == '.DS_Store': continue
  os.rename(path + file, final_path + file)

path = 'dataset/hand_dataset/test_dataset/test_data/images/'
counter = 40000
for file in sorted(glob.glob(path + '*.jpg')):
  if file == '._.DS_Store' or file == '.DS_Store': continue
  os.rename(file, final_path + str(counter) + '.jpg')
  counter += 1

!rm -rf dataset/hand_dataset

In the case of the HandsOverFace dataset we just import it from the support repository and move it into the images folder.

As the last step of this organization we also move all the annotations from the photos into the dataset/labels/ directory,
to have a easier to understand structure of the data.

In [7]:
# Finally we move the labels we imported to the labels folder
final_path = 'dataset/labels/'
initial_path = 'dataset_CV/labels/'
for file in os.listdir(initial_path):
  os.rename(initial_path + file, final_path + file)



The next cell serves the purpose of checking that we moved all the data correctly.
First we check that we have the same number of images and annotation files by printing the number of files in each one,
and if an annotation file is missing then we print the name of its respective image.

In [ ]:
print(len(os.listdir('dataset/images/')))
print(len(os.listdir('dataset/labels/')))

path_1 = 'dataset/images/'
path_2 = 'dataset/labels/'

for file in os.listdir(path_1):
    txt_name = file[:-4] + '.txt'
    if txt_name not in os.listdir(path_2):
        print(txt_name + ' missing')


7728
8304


Now we want to create the validation and test set. We just take 1000 random samples for validation and 800 for test.

In [9]:
!mkdir dataset/validation/ dataset/validation/images/ dataset/validation/labels/
!mkdir dataset/test/ dataset/test/images dataset/test/labels

random.seed(10000)

images_path = 'dataset/images/'
labels_path = 'dataset/labels/'
validation_images = 'dataset/validation/images/'
validation_labels = 'dataset/validation/labels/'
test_images = 'dataset/test/images/'
test_labels = 'dataset/test/labels/'

# Here we populate the validation set
for i in range(1000):
  list = sorted(os.listdir(images_path))
  n = random.randint(0, len(list) - 1)
  #print(list[n][:-4] + ' -> validation')
  os.rename(images_path + list[n], validation_images + list[n])
  os.rename(labels_path + list[n][:-4] + '.txt', validation_labels + list[n][:-4] + '.txt')

# Here we populate the test set


for i in range(800):
  list = sorted(os.listdir(images_path))
  n = random.randint(0, len(list) - 1)
  #print(list[n][:-4] + ' -> test')
  os.rename(images_path + list[n], test_images + list[n])
  os.rename(labels_path + list[n][:-4] + '.txt', test_labels + list[n][:-4] + '.txt')
  
print(len(os.listdir(validation_images)))
print(len(os.listdir(validation_labels)))
print(len(os.listdir(test_images)))
print(len(os.listdir(test_labels)))

VOC2010_945 -> validation
Buffy_249 -> validation
14097 -> validation
27060 -> validation
13013 -> validation
1004 -> validation
5057 -> validation
VOC2010_1171 -> validation
Inria_168 -> validation
35081 -> validation
25018 -> validation
VOC2010_839 -> validation
Movie_apollo13_129 -> validation
10089 -> validation
Buffy_38 -> validation
12024 -> validation
VOC2010_909 -> validation
40358 -> validation
12023 -> validation
Inria_209 -> validation
VOC2010_999 -> validation
6048 -> validation
VOC2010_1341 -> validation
VOC2010_162 -> validation
33077 -> validation
27029 -> validation
22096 -> validation
Buffy_78 -> validation
11064 -> validation
Poselet_283 -> validation
32064 -> validation
19021 -> validation
Movie_4weds_71 -> validation
VOC2007_456 -> validation
Poselet_217 -> validation
40795 -> validation
27077 -> validation
3099 -> validation
2045 -> validation
40536 -> validation
VOC2007_729 -> validation
11086 -> validation
1051 -> validation
Poselet_373 -> validation
6032 -> vali

In [11]:
# hands over face
path = 'dataset_CV/hands_over_face/'
for file in os.listdir(path):
  os.rename(path + file, final_path + file)

!rm -rf dataset_CV

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_CV/hands_over_face/'

### 3) Check conversions
Now to check if the conversion result is correct by visualizing random samples for each subset of the dataset and drawing the converted bounding box(es) onto it.<br>

In [ ]:
datasets_extr = ["dataset/","dataset/test/", "dataset/validation/"]

for dir_path in datasets_extr:
    # randomly select an image
    rnd_img = random.choice(os.listdir(dir_path + "images/"))
    print(rnd_img)

    # open the image
    img = Image.open(dir_path + "images/" + rnd_img)

    # create figure and axes
    fig, ax = plt.subplots(dpi=163)

    # display the image
    ax.imshow(img)

    #covert line string into ndarray
    bbox_yolo_coords = []
    with open(dir_path + "labels/" + rnd_img[:len(rnd_img)-4] + ".txt") as txt:

        lines = txt.read().splitlines()
        for line in lines:
            bbox_yolo_coords.append(np.fromstring(line[1:], dtype=float, sep=' ', count=4))

    # de-normalize coordinates and draw bbox onto img
    for box in bbox_yolo_coords:
        box[0] *= float(img.width)
        box[2] *= float(img.width)
        box[1] *= float(img.height)
        box[3] *= float(img.height)
        ax.add_patch(patches.Rectangle((box[0]-box[2]/2., box[1]-box[3]/2.), width=box[2], height=box[3], linewidth=1, edgecolor='r', facecolor='none'))


### 4) Last steps before training
Now we need to setup last things to train the yolo model. We will follow the following steps:
- Copy images and labels into a directory system suitable to yolov5

In [ ]:
dirs = ["./images/", "./labels/", "./images/train/", "./labels/train/", "./images/val/", "./labels/val/", "./images/test/", "./labels/test/"]
for dir_ in dirs:
    if not os.path.exists(dir_):
        os.mkdir(dir_)

# move dataset into directories "images" and "labels" which are located in same position as the notebook
subsets = ["train/", "val/", "test/"]
for i in range(3):
    for img_name in os.listdir(datasets_extr[i] + "images/"):
        old_img_path = datasets_extr[i] + "images/" + img_name
        old_lbl_path = datasets_extr[i] + "labels/" + img_name[:len(img_name)-3] + "txt"
        new_img_path = dirs[0] + subsets[i] + img_name
        new_lbl_path = dirs[1] + subsets[i] + img_name[:len(img_name)-3] + "txt"

        # copy img if it's not already there
        if not os.path.exists(new_img_path):
            shutil.copyfile(old_img_path, new_img_path)
        if not os.path.exists(new_lbl_path):
            shutil.copyfile(old_lbl_path, new_lbl_path)

- Create dataset location _yaml_ file (In our case move the premade one into the right directory)

In [ ]:
# move data.yaml
new_data_yaml = "./yolov5/data/hand_data.yaml"
if os.path.exists(new_data_yaml):
    os.remove(new_data_yaml)
shutil.copyfile("./hand_data.yaml", new_data_yaml)


print("------------------ hand_data.yaml ---------------------")
with open(new_data_yaml) as f:
    lines = f.read().splitlines()
    for line in lines:
        print("\t" + line)
print("-------------------------------------------------------")

- Setup hyperparameters file, which will contain all usual parameters used when training a neural network (Same as before, we will move the premade one into the right directory)<br>
  We are just going to use the default one

In [ ]:
hyperparams_yaml = "./yolov5/data/hyps/hyp.scratch-med.yaml"
print("-------------------------------------------------- hyp.scratch-med.yaml -----------------------------------------------------")
with open(hyperparams_yaml) as f:
    lines = f.read().splitlines()
    for line in lines:
        print("\t" + line)
print("-----------------------------------------------------------------------------------------------------------------------------")

- A custom network architecture can also be specified by specifing with _-cfg_ argument the custom network architecture file.<br>
  Below there is and example taken from a default yolo network architecture

In [ ]:
net_architecture_yaml = "./yolov5/models/yolov5l.yaml"
print("------------------------------------------------------ yolov5m.yaml --------------------------------------------------------")
with open(net_architecture_yaml) as f:
    lines = f.read().splitlines()
    for line in lines:
        print("\t" + line)
print("-----------------------------------------------------------------------------------------------------------------------------")

### 5) Training the yolo model

In [ ]:
# 313 could be a good batch size (1,13,313,4069)
#!cd yolov5
!python yolov5/train.py --img 640 --cfg yolov5s.yaml --hyp hyp.scratch-med.yaml --batch 20 --epochs 60 --data hand_data.yaml --weights ' ' --workers 8 --name yolo_hand_localization

Now to download the results: first create a zip file, then download it

In [ ]:
!zip -r ./training_result_runs.zip ./yolov5/runs